In [1]:
from keithley2600 import Keithley2600
from pmu_som import communicator, pmu_ctrl, pmu_cal
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd


try:
    del(p.c)
except:
    pass

k = Keithley2600('GPIB0::17::INSTR', visa_library='')
#k.smua.source.output = k.smua.OUTPUT_OFF

k.smua.reset()
k.smub.reset()

# p = communicator.communicator(False)
p = pmu_ctrl.pmu()

p.channels[0].change_pmu_reg({
    "CH EN" : 0
})

k.display.screen = k.display.SMUA
k.display.smua.measure.func = k.display.MEASURE_DCAMPS

k.smua.measure.nplc = 1
k.smua.measure.autorangei = k.smua.AUTORANGE_ON
k.smua.measure.autorangev = k.smua.AUTORANGE_ON

k.smua.source.limiti = ((2e-3)*2)
k.smua.source.limitv = 20
k.smua.source.func = k.smua.OUTPUT_DCVOLTS
k.smua.source.levelv = 0
k.smua.source.output = k.smua.OUTPUT_ON

p.change_sys_ctrl({
    "DUTGND/CH" : 1,
    "INT10K"    : 1,
    "GAIN"      : 2,
    "TMP ENABLE": 1,
    "TMP"       : 3 # Thermal shutdown at 100°C
    })


p.channels[0].change_pmu_reg({
    "CH EN" : 1, # enable the channel
    "FORCE" : 0, # force voltage clamp current
    "SS0"  : 1,
    "SF0"  : 1,
    "FIN"  : 1,
    "C"    : 3,
    "CL"   : 1
    })


overlap = 0.5
num =10

measure_points = np.linspace(round(0x8000-(0x8000*overlap)), 0xFFFF, num)
val = 5
offset_dac = 0xA492
VREF = 4.95
DAC_C = round(((val * (2**16) / VREF) + (3.5*offset_dac)) / 4.5)



measure_delay = 0.1


## This code is only for clamp low
## Writing the DAC CODE equivalent to 5V output at "FIN U"
p.channels[0].write_dac(
    data = 0xDAC_C,
    dac_reg = "FIN_U",
    reg = "X1"
    )

## Writing the clamp high as FFFF
p.channels [0].write_dac(
    data = 0x0000,
    dac_reg = "CLL_I",
    reg = "X1"
)

dac_regi = "CLH_I"
clamp_measured_values = []
clamp_expected_values = []

## Clamping Function (Function for Clamping)
## Clamping from 0x7fff(32767) to 0x000 for an input voltage of 5v (FIN U)



for d in measure_points :
    p.channels[0].write_dac(
        data = round(d),
        dac_reg = dac_regi,
        reg = "X1"
    )
    sleep(measure_delay)
    clamp_measured = k.smua.measure.i()
    clamp_measured_values.append(clamp_measured)
    val = d
    rsens = 500
    MI_GAIN =10
    clamp_expected = (4.5 * VREF * (val-(2**15))) / ((2**16)*rsens*MI_GAIN)
    clamp_expected_values.append(clamp_expected)
    print("Expected Clamping :",clamp_expected)
    print("Measured Clamping :",clamp_measured)
    
    

p.channels[0].change_pmu_reg({
    "CH EN" :0
})

k.smua.source.output = k.smua.OUTPUT_OFF
df = pd.DataFrame({
    "Measured Clamping " : clamp_measured_values,
    "Expected Clamping " : clamp_expected_values 
})

print(df)
 

Expected Clamping : -0.0011137500000000002
Measured Clamping : 0.000988751
Expected Clamping : -0.0007425075531005859
Measured Clamping : 0.000612653
Expected Clamping : -0.00037126510620117184
Measured Clamping : 0.000236308
Expected Clamping : -2.2659301757730066e-08
Measured Clamping : -0.00014066
Expected Clamping : 0.0003712197875976564
Measured Clamping : -0.000517952
Expected Clamping : 0.0007424622344970702
Measured Clamping : -0.000895038
Expected Clamping : 0.0011137046813964847
Measured Clamping : -0.00127181
Expected Clamping : 0.001484947128295899
Measured Clamping : -0.00164873
Expected Clamping : 0.001856189575195313
Measured Clamping : -0.00202565
Expected Clamping : 0.0022274320220947268
Measured Clamping : -0.00240291
   Measured Clamping   Expected Clamping 
0            0.000989       -1.113750e-03
1            0.000613       -7.425076e-04
2            0.000236       -3.712651e-04
3           -0.000141       -2.265930e-08
4           -0.000518        3.712198e-04
5 

In [2]:
pmu_cal.measureCL_I(p, k, 0, range = "2m", voltage= 5, clamp_side='H', overlap=0.5, num=10, measure_delay = 0.1)

(array([-1.11375000e-03, -7.42507553e-04, -3.71265106e-04, -2.26593018e-08,
         3.71219788e-04,  7.42462234e-04,  1.11370468e-03,  1.48494713e-03,
         1.85618958e-03,  2.22743202e-03]),
 array([ 0.00098884,  0.00061276,  0.00023639, -0.00014058, -0.00051788,
        -0.00089499, -0.00127159, -0.00164852, -0.00202552, -0.00240274]))

In [13]:
pmu_cal.measureCL_U(p, k, 0, range = "2m", current_range_factor=0.5, clamp_side='H', overlap=0.5, num=10, measure_delay = 0.1)

(array([-5.56867447e+00, -3.71246223e+00, -1.85625000e+00, -3.77655029e-05,
         1.85617447e+00,  3.71238670e+00,  5.56859894e+00,  7.42481117e+00,
         9.28102341e+00,  1.11372356e+01]),
 array([-5.51628 , -3.64343 , -1.77027 ,  0.103221,  1.97654 ,  3.84971 ,
         5.72328 ,  7.59483 ,  9.4675  , 11.3407  ]))

In [9]:
pmu_cal.measureCL_U((pmu, keithley, ch, range, current_range_factor := 0.5, clamp_side :='H', overlap :=0.5, num :=100, measure_delay := 0.1):

SyntaxError: invalid syntax (1686110656.py, line 1)